<a href="https://colab.research.google.com/github/gurusanand/prodigy-recipes/blob/master/Names_Copy_of_MB_task_using_LLama2_13B_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook needs T4 GPU. Please select the GPU by selecting the menu
**Runtime->Change runtime type**

**Please upload the pdf files in Colab before running any cells in this notebook**

In [2]:
print("Upload your pdf files to Colab")
from google.colab import files
files.upload()

Upload your pdf files to Colab


Saving loan.pdf to loan.pdf


{'loan.pdf': b'%PDF-1.7\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n<</Type/Catalog/Pages 2 0 R/Lang(en) /StructTreeRoot 316 0 R/MarkInfo<</Marked true>>/Metadata 5399 0 R/ViewerPreferences 5400 0 R>>\r\nendobj\r\n2 0 obj\r\n<</Type/Pages/Count 122/Kids[ 3 0 R 53 0 R 55 0 R 57 0 R 73 0 R 76 0 R 78 0 R 80 0 R 82 0 R 84 0 R 86 0 R 88 0 R 90 0 R 92 0 R 94 0 R 96 0 R 98 0 R 100 0 R 102 0 R 104 0 R 106 0 R 108 0 R 110 0 R 112 0 R 114 0 R 116 0 R 118 0 R 120 0 R 122 0 R 124 0 R 126 0 R 128 0 R 130 0 R 132 0 R 134 0 R 136 0 R 138 0 R 140 0 R 142 0 R 144 0 R 146 0 R 148 0 R 150 0 R 152 0 R 154 0 R 156 0 R 158 0 R 160 0 R 162 0 R 164 0 R 166 0 R 168 0 R 170 0 R 172 0 R 174 0 R 176 0 R 178 0 R 180 0 R 182 0 R 184 0 R 186 0 R 188 0 R 190 0 R 192 0 R 194 0 R 196 0 R 198 0 R 200 0 R 205 0 R 207 0 R 209 0 R 211 0 R 213 0 R 215 0 R 217 0 R 219 0 R 221 0 R 223 0 R 225 0 R 227 0 R 229 0 R 231 0 R 233 0 R 235 0 R 237 0 R 239 0 R 241 0 R 243 0 R 245 0 R 247 0 R 249 0 R 251 0 R 253 0 R 255 0 R 257 0 R 259 0 R 261 

In [3]:
!nvidia-smi

Sat Oct 28 08:30:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installations and Imports

In [4]:
!pip install -q langchain transformers accelerate bitsandbytes PyPDF2 optimum auto-gptq

from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline, PromptTemplate,  LLMChain
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, pipeline
import json
import os
import torch
import PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 36.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 

# Model and Pipeline Setup

In [5]:
%%time
#Tried models -> mistralai/Mistral-7B-Instruct-v0.1 NousResearch/Llama-2-7b-chat-hf TheBloke/Mistral-7B-Instruct-v0.1-GPTQ TheBloke/Llama-2-13B-chat-GPTQ Trelis/Llama-2-7b-chat-hf-hosted-inference-8bit
#Current model -> TheBloke/Llama-2-13B-chat-GPTQ
#Havent tested yet ->TheBloke/vicuna-33B-GPTQ
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-13B-chat-GPTQ")
model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-13B-chat-GPTQ",
                                             device_map='auto',
                                             torch_dtype=torch.float16)
#load_in_4bit=True,
  #                                           bnb_4bit_quant_type="nf4",
   #                                          bnb_4bit_compute_dtype=torch.float16)

CPU times: user 19.1 s, sys: 16.6 s, total: 35.7 s
Wall time: 4min 46s


In [6]:
#Pipelne setup
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

# Prompts and LLM Chain Setup

In [7]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an useful AI Assistant.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.Just say you don't know and you are sorry!"""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [15]:
#individual entity prompts
system_prompt = "You are an expert in identification of requested data from natural language text. You are terse and accurate in your replies. You do not infer. Your response should not be more than 1000 words"
#bn_instruction = "Identify the borrower from the passage:\n\n {text} \n\n If there is no borrower, provide the response 'No data' as output. Critically validate your response once."
 #bn_instruction = "Identify the Paying Agent from the passage:\n\n {text} \n\n If there is no Paying Agent, provide the response 'No data' as output. Critically validate your response once."
#cc_instruction = "Without inferring, extract the currency from the passage:\n\n {text} \n\n If there is no currency, provide the response 'No data' as output. Critically validate your response once."
#sd_instruction = "Identify the agreement date or amendment date or document date from the passage:\n\n {text} \n\n If there is no agreement date or amendment date or document date, provide the response 'No data' as output. Critically validate your response once and rectify incorrect date."
names_instruction="Identify all names, from the passage:\n\n {text} \n\n If there is no  names provide the response 'No data' as output. Critically validate your response once."
#bn_template = get_prompt(bn_instruction, system_prompt)
#cc_template = get_prompt(cc_instruction, system_prompt)
#sd_template = get_prompt(sd_instruction, system_prompt)
names_template = get_prompt(names_instruction, system_prompt)
#print(bn_template)
#print(cc_template)
#print(sd_template)

#consolidation prompts
#bn_c_instruction = "Identify the Borrower from the loan agreement data:\n\n {text} \n\n"
#cc_c_instruction = "If there is any currency mentioned in the loan agreement data, please identify. Do not infer. Here is the loan agreement  data:\n\n {text} \n\\n Critically validate your response once and make corrections"
 #cc_c_instruction = "Identify the currency from the log data:\n\n {text} \n\n"
 #cc_c_instruction = "Identify the frequently occuring currency code from the log data:\n\n {text} \n\n"
#sd_c_instruction = "Identify the recurring agreement date and/or amendment date and/or document date from the loan agreement data:\n\n {text} \n\n"
names_c_instruction = "Identify  all the names from the loan agreement data:\n\n {text} \n\n"
#bn_c_template = get_prompt(bn_c_instruction, system_prompt)
#cc_c_template = get_prompt(cc_c_instruction, system_prompt)
#sd_c_template = get_prompt(sd_c_instruction, system_prompt)
names_c_template=get_prompt(names_c_instruction, system_prompt)
#print(bn_c_template)
#print(cc_c_template)
#print(sd_c_template)

In [16]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 1024, 'top_k' :1})

#bn_prompt = PromptTemplate(template=bn_template, input_variables=["text"])
#cc_prompt = PromptTemplate(template=cc_template, input_variables=["text"])
#sd_prompt = PromptTemplate(template=sd_template, input_variables=["text"])
names_prompt = PromptTemplate(template=names_template, input_variables=["text"])

#bn_llm_chain = LLMChain(prompt=bn_prompt, llm=llm, verbose = False)
#cc_llm_chain = LLMChain(prompt=cc_prompt, llm=llm, verbose = False)
#sd_llm_chain = LLMChain(prompt=sd_prompt, llm=llm, verbose = False)
names_llm_chain = LLMChain(prompt=names_prompt, llm=llm, verbose = False)

#bn_c_prompt = PromptTemplate(template=bn_c_template, input_variables=["text"])
#cc_c_prompt = PromptTemplate(template=cc_c_template, input_variables=["text"])
#sd_c_prompt = PromptTemplate(template=sd_c_template, input_variables=["text"])
names_c_prompt = PromptTemplate(template=names_c_template, input_variables=["text"])

#bn_c_llm_chain = LLMChain(prompt=bn_c_prompt, llm=llm, verbose = False)
#cc_c_llm_chain = LLMChain(prompt=cc_c_prompt, llm=llm, verbose = False)
#sd_c_llm_chain = LLMChain(prompt=sd_c_prompt, llm=llm, verbose = False)
names_c_llm_chain = LLMChain(prompt=names_c_prompt, llm=llm, verbose = False)

# Data Augmentation

In [23]:
#data augmentation using summarization - This is used in the main code
pipe_s = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 1024,
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
llm_s = HuggingFacePipeline(pipeline = pipe_s, model_kwargs = {'temperature':0.7,'max_length': 1024, 'top_k' :1})
system_prompt_s = "You are an expert in selecting only the unique names from the English text. You never miss out any important details."
 #system_prompt_s = "You are an expert in summarizing English text. You never miss out any important details."
s_instruction = "Take only unique names from the following page from a loan agreement document in the page:\n\n {text} \n\n Mention  all the unique names explicitly."
 #s_instruction = "Summarize the following page from a loan agreement document in a single paragraph:\n\n {text} \n\n Mention the borrower, currency (if present),  agreement date and all the organization names explicitly."
s_template = get_prompt(s_instruction, system_prompt_s)
s_prompt = PromptTemplate(template=s_template, input_variables=["text"])
s_llm_chain = LLMChain(prompt=s_prompt, llm=llm_s, verbose = False)

In [24]:
#NO NEED TO EXECUTE THIS CELL
#data augmentation using paraphrasing - not used in main extraction currently
'''
pipe_p = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 1024,
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
llm_p = HuggingFacePipeline(pipeline = pipe_p, model_kwargs = {'temperature':0.7,'max_length': 1024, 'top_k' :1})
system_prompt_p = "You are an expert in paraphrasing english text. You dont leave out any factual details from the input text while paraphrasing."
p_instruction = "Paraphrase the following page content from a loan agreement date:\n\n {text} \n\n"
p_template = get_prompt(p_instruction, system_prompt_p)
p_prompt = PromptTemplate(template=p_template, input_variables=["text"])
p_llm_chain = LLMChain(prompt=p_prompt, llm=llm_p, verbose = False)
'''

'\npipe_p = pipeline("text-generation",\n                model=model,\n                tokenizer= tokenizer,\n                torch_dtype=torch.float16,\n                device_map="auto",\n                max_new_tokens = 1024,\n                do_sample=True,\n                top_k=10,\n                num_return_sequences=1,\n                eos_token_id=tokenizer.eos_token_id\n                )\nllm_p = HuggingFacePipeline(pipeline = pipe_p, model_kwargs = {\'temperature\':0.7,\'max_length\': 1024, \'top_k\' :1})\nsystem_prompt_p = "You are an expert in paraphrasing english text. You dont leave out any factual details from the input text while paraphrasing."\np_instruction = "Paraphrase the following page content from a loan agreement date:\n\n {text} \n\n"\np_template = get_prompt(p_instruction, system_prompt_p)\np_prompt = PromptTemplate(template=p_template, input_variables=["text"])\np_llm_chain = LLMChain(prompt=p_prompt, llm=llm_p, verbose = False)\n'

# Main Extraction

In [25]:
%%time
#SyndLoan Sample 1.pdf
#SyndLoan Sample 2.pdf
#SyndLoan Sample 3.pdf
#SyndLoan Sample 4.pdf
bn_output=""
cc_output=""
sd_output=""
names_output=""

filename=input("Provide the file name with extension:")
pagemax=int(input("Provide the number of pages to be scanned:"))

with open(os.path.join('/content/', filename),'rb') as f:
  print(filename)
  reader = PyPDF2.PdfReader(filename)
  #read_pdf = PyPDF2.PdfReader(reader)
  number_of_pages = len(reader.pages)
  for page_number in range(pagemax):
      page = reader.pages[page_number]
      print("Processing Page ",page_number+1)
      page_content = page.extract_text()
      #print(page_content)
      summary=s_llm_chain.run(page_content)
      #print(summary)
      #bn_response = bn_llm_chain.run(summary)
      #cc_response = cc_llm_chain.run(summary)
      #sd_response = sd_llm_chain.run(summary)

      #print(bn_response)
      #print(cc_response)
      #print(sd_response)
      print("names_response")
      #names_response=names_llm_chain.run({"input": summary})
      names_response=names_llm_chain.run(summary)
      #print(names_response)
      #print("After names_response ")
      #bn_output=bn_output+"\n"+bn_response
      #cc_output=cc_output+"\n"+cc_response
      #sd_output=sd_output+"\n"+sd_response
      names_output=names_output+"\n"+names_response
      #print("bn_output",bn_output)
      #print("cc_output",cc_output)
      #print("sd_output",sd_output)
      print("names_output",names_output)
  #bn_output1=bn_output.replace("No data", "")
  #cc_output1=cc_output.replace("No data", "")
  #sd_output1=sd_output.replace("No data", "")
  names_output1=names_output.replace("No data", "")
  #bn_c_response = bn_c_llm_chain.run(bn_output1)
  #cc_c_response = cc_c_llm_chain.run(cc_output1)
  #sd_c_response = sd_c_llm_chain.run(sd_output1)
  names_c_response = names_c_llm_chain.run(names_output1)
  #print('bn_c_response.strip',bn_c_response.strip())
  #print('cc_c_response.strip',cc_c_response.strip())
  #print('sd_c_response.strip',sd_c_response.strip())
  print('All Names :',names_c_response.strip())
  print("\n")

Provide the file name with extension:loan.pdf
Provide the number of pages to be scanned:5
loan.pdf
Processing Page  1
names_response


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


names_output 
  Sure, I'd be happy to help! Based on the passage provided, there are the following unique names mentioned:

1. AMD Saxony Manufacturing Gmbh
2. AMD Saxony Holding Gmbh
3. Dresdner Bank AG
4. Dresdner Bank Luxembourg S.A.

Therefore, the answer is:

No data

Note: I have critically validated my response to ensure accuracy and there is no inferencing done in my analysis.
Processing Page  2
names_response


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


names_output 
  Sure, I'd be happy to help! Based on the passage provided, there are the following unique names mentioned:

1. AMD Saxony Manufacturing Gmbh
2. AMD Saxony Holding Gmbh
3. Dresdner Bank AG
4. Dresdner Bank Luxembourg S.A.

Therefore, the answer is:

No data

Note: I have critically validated my response to ensure accuracy and there is no inferencing done in my analysis.
  Based on the provided passage, the following are the unique names mentioned:

1. AMD (Advanced Micro Devices)
2. Saxonia
3. Holding
4. Sponsors
5. Guaranty
6. Subordination
7. Inc.
8. O'Melveny & Xxxxx LLP
9. Xxxx, Stiefenhofer & Xxxx
10. Xxxxx & XxXxxxxx/Doser Amereller Xxxxx
11. White & Case, Xxxxxxxxx

There is no other name mentioned in the passage.

Please note that some of these names are mentioned multiple times, but I have only listed each name once.
Processing Page  3
names_response


...

names_output 
  Sure, I'd be happy to help! Based on the passage provided, there are the following unique names mentioned:

1. AMD Saxony Manufacturing Gmbh
2. AMD Saxony Holding Gmbh
3. Dresdner Bank AG
4. Dresdner Bank Luxembourg S.A.

Therefore, the answer is:

No data

Note: I have critically validated my response to ensure accuracy and there is no inferencing done in my analysis.
  Based on the provided passage, the following are the unique names mentioned:

1. AMD (Advanced Micro Devices)
2. Saxonia
3. Holding
4. Sponsors
5. Guaranty
6. Subordination
7. Inc.
8. O'Melveny & Xxxxx LLP
9. Xxxx, Stiefenhofer & Xxxx
10. Xxxxx & XxXxxxxx/Doser Amereller Xxxxx
11. White & Case, Xxxxxxxxx

There is no other name mentioned in the passage.

Please note that some of these names are mentioned multiple times, but I have only listed each name once.
  Sure! Here are all the unique names explicitly mentioned in the text:

1. AMD Saxony Manufacturing GMBH
2. AMD Saxony Holding GMBH
3. Dresdner Ba

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


names_output 
  Sure, I'd be happy to help! Based on the passage provided, there are the following unique names mentioned:

1. AMD Saxony Manufacturing Gmbh
2. AMD Saxony Holding Gmbh
3. Dresdner Bank AG
4. Dresdner Bank Luxembourg S.A.

Therefore, the answer is:

No data

Note: I have critically validated my response to ensure accuracy and there is no inferencing done in my analysis.
  Based on the provided passage, the following are the unique names mentioned:

1. AMD (Advanced Micro Devices)
2. Saxonia
3. Holding
4. Sponsors
5. Guaranty
6. Subordination
7. Inc.
8. O'Melveny & Xxxxx LLP
9. Xxxx, Stiefenhofer & Xxxx
10. Xxxxx & XxXxxxxx/Doser Amereller Xxxxx
11. White & Case, Xxxxxxxxx

There is no other name mentioned in the passage.

Please note that some of these names are mentioned multiple times, but I have only listed each name once.
  Sure! Here are all the unique names explicitly mentioned in the text:

1. AMD Saxony Manufacturing GMBH
2. AMD Saxony Holding GMBH
3. Dresdner Ba

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


All Names : Based on the provided loan agreement data, the following are the unique names mentioned:

1. AMD Saxony Manufacturing Gmbh
2. AMD Saxony Holding Gmbh
3. Dresdner Bank AG
4. Dresdner Bank Luxembourg S.A.

There is no other name mentioned in the text.


CPU times: user 1min 12s, sys: 2.7 s, total: 1min 15s
Wall time: 1min 27s
